In [1]:
# Import and run a trained model 
import numpy as np
import tensorflow.compat.v1 as tf
from sklearn.metrics import confusion_matrix
import pandas as pd

In [2]:
def get_test_batch(XTest, yTest, batchSize):
    #a generator that slices and returns batchSize of XTrain and yTrain instances from top down
    counter = 0;
    while True:
        if counter >= len(yTest):
            break;
        returnXTest = XTest[counter:counter + batchSize];
        returnYTest = yTest[counter:counter + batchSize];
        counter = counter + batchSize;
        yield returnXTest, returnYTest

In [3]:
raw_x_test = np.load('numpyData/raw_x_test.npy')
raw_y_test = np.load('numpyData/raw_y_test.npy')
y_test_dummy = np.zeros((260, 3))

batch_size = 260
latent_size = 100
sequence_length = 64
num_classes = 3

In [4]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('savedModels/GAN/NLP_embedding_80_best/recall0_0.7272727272727273recall2_0.7666666666666667_epoch_337_SSL_GAN.ckpt.meta')
    saver.restore(sess, 'savedModels/GAN/NLP_embedding_80_best/recall0_0.7272727272727273recall2_0.7666666666666667_epoch_337_SSL_GAN.ckpt')
    
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name('x:0')
    z = graph.get_tensor_by_name('z:0')
    label = graph.get_tensor_by_name('label:0')
    labeled_mask = graph.get_tensor_by_name('labeled_mask:0')
    dropout_rate = graph.get_tensor_by_name('dropout_rate:0')
    is_training = graph.get_tensor_by_name('is_training:0')
    
    logits = graph.get_tensor_by_name('Discriminator/scores:0')
    conv = graph.get_tensor_by_name('Discriminator/conv:0')
    pool = graph.get_tensor_by_name('Discriminator/pool:0') 
    
    test_logits_all = []
    test_label_all = []
    for batch_x, batch_y in get_test_batch(raw_x_test, y_test_dummy, batch_size):
        mask = np.ones(len(batch_y)) # Mark all as labeled
        batch_z = np.random.normal(0, 1, size = (len(batch_y), latent_size))

        test_feed_dictionary = {x: batch_x,
                                z: batch_z,
                                label: batch_y,
                                labeled_mask: mask,
                                dropout_rate: 0.0,
                                is_training: False}
        test_logits = logits.eval(feed_dict=test_feed_dictionary)
        print('conv shape')
        print(conv.shape)
        print('pool shape')
        print(pool.shape)
        
        
        
        test_logits_all.append(test_logits)
        test_label_all.append(batch_y)
    test_logits_all = np.concatenate(test_logits_all)
    test_label_all = np.concatenate(test_label_all)
    preds = np.argmax(test_logits_all, axis=1)

INFO:tensorflow:Restoring parameters from savedModels/GAN/NLP_embedding_80_best/recall0_0.7272727272727273recall2_0.7666666666666667_epoch_337_SSL_GAN.ckpt
conv shape
(?, 62, 1, 3)
pool shape
(?, 1, 1, 3)


In [5]:
preds.shape

(260,)

In [6]:
preds

array([2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0,
       2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0,
       2, 0, 1, 0, 0, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0,
       0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2,
       0, 1, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0,
       2, 2, 1, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 0,
       2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2], dtype=int64)

In [7]:
confusion_matrix(np.argmax(raw_y_test, axis=1), preds)

array([[ 32,   0,  12,   0],
       [  1,   0,   0,   0],
       [ 19,   2,  69,   0],
       [ 18,   6, 101,   0]], dtype=int64)

In [8]:
import pandas as pd

df = pd.read_csv('albania.csv')
label = []
for pred in preds:
    if pred == 0:
        label.append('Building')
    if pred == 1:
        label.append('Infrastructure')
    if pred == 2:
        label.append('Resilience')

In [9]:
df

,sentence,label
0,Albania’s search for quake victims ends as dea...,Resilience
1,Albania’s PM announces end of operation launch...,Resilience
2,The search-and-rescue operation for earthquake...,Resilience
3,Rama said on Saturday that preliminary figures...,Building
4,"About 2,000 people were injured.",Resilience
...,...,...
255,"On 12 December, Italian Foreign Minister Luigi...",Other
256,Following the completion of the earthquake dam...,Other
257,A Post-Disaster Needs Assessment (PDNA) was es...,Other
258,On 10 January the EU announced that it had org...,Other


In [10]:
label

['Resilience',
 'Resilience',
 'Building',
 'Building',
 'Resilience',
 'Resilience',
 'Resilience',
 'Resilience',
 'Building',
 'Building',
 'Building',
 'Resilience',
 'Resilience',
 'Resilience',
 'Resilience',
 'Resilience',
 'Building',
 'Resilience',
 'Building',
 'Resilience',
 'Resilience',
 'Building',
 'Resilience',
 'Resilience',
 'Building',
 'Resilience',
 'Resilience',
 'Building',
 'Building',
 'Resilience',
 'Building',
 'Building',
 'Resilience',
 'Resilience',
 'Resilience',
 'Building',
 'Resilience',
 'Building',
 'Building',
 'Building',
 'Resilience',
 'Resilience',
 'Resilience',
 'Building',
 'Resilience',
 'Building',
 'Infrastructure',
 'Building',
 'Building',
 'Resilience',
 'Resilience',
 'Building',
 'Building',
 'Resilience',
 'Building',
 'Resilience',
 'Resilience',
 'Resilience',
 'Resilience',
 'Resilience',
 'Resilience',
 'Resilience',
 'Building',
 'Resilience',
 'Resilience',
 'Building',
 'Building',
 'Resilience',
 'Building',
 'Building',
 'Re

In [11]:
df['GAN'] = label

In [12]:
df

,sentence,label,GAN
0,Albania’s search for quake victims ends as dea...,Resilience,Resilience
1,Albania’s PM announces end of operation launch...,Resilience,Resilience
2,The search-and-rescue operation for earthquake...,Resilience,Building
3,Rama said on Saturday that preliminary figures...,Building,Building
4,"About 2,000 people were injured.",Resilience,Resilience
...,...,...,...
255,"On 12 December, Italian Foreign Minister Luigi...",Other,Building
256,Following the completion of the earthquake dam...,Other,Resilience
257,A Post-Disaster Needs Assessment (PDNA) was es...,Other,Resilience
258,On 10 January the EU announced that it had org...,Other,Resilience


In [13]:
confusion_matrix(df['label'], df['GAN'])

array([[ 32,   0,   0,  12],
       [  1,   0,   0,   0],
       [ 18,   6,   0, 101],
       [ 19,   2,   0,  69]], dtype=int64)

In [14]:
df

,sentence,label,GAN
0,Albania’s search for quake victims ends as dea...,Resilience,Resilience
1,Albania’s PM announces end of operation launch...,Resilience,Resilience
2,The search-and-rescue operation for earthquake...,Resilience,Building
3,Rama said on Saturday that preliminary figures...,Building,Building
4,"About 2,000 people were injured.",Resilience,Resilience
...,...,...,...
255,"On 12 December, Italian Foreign Minister Luigi...",Other,Building
256,Following the completion of the earthquake dam...,Other,Resilience
257,A Post-Disaster Needs Assessment (PDNA) was es...,Other,Resilience
258,On 10 January the EU announced that it had org...,Other,Resilience


In [15]:
df.to_csv('albania_GAN_predictions.csv')